In [1]:
from google.colab import drive 


drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
DRIVE_PATH = '/content/gdrive/My Drive/CS/SI/part2/text-generator/'

In [0]:
import pandas as pd

df = pd.read_csv(DRIVE_PATH + "spanish_emojis.csv")

In [4]:
df.head()

,Unnamed: 0,emojis,observations
0,300,😭,? en serio han cancelado tambien quantico
1,301,😂,si on mo
2,302,😝,se duerme relooo
3,303,🤗,ahi te podes dar cuenta que diferentes somos y...
4,304,😪,625 pero la tipi esta jugando desde la 5


In [5]:
observations = df['observations'].values
len(observations)

151743

In [6]:
observations[0]

'? en serio han cancelado tambien quantico'

In [7]:
sentences = []
for sentence in observations:
    if len(sentence.split()) > 18:
      sentences.append(sentence.split())
      
print(len(sentences))

13336


In [0]:
sentences = sentences[:10000]

In [9]:
flatten_word_simbols = [word_simbol for sublist in sentences for word_simbol in sublist]

len(flatten_word_simbols)

220482

In [10]:
import collections

cnt = collections.Counter(flatten_word_simbols)
print("Vocab size = ", len(cnt))

Vocab size =  19779


In [0]:
vocab_keys = cnt.most_common()

In [12]:
vocab_keys[-1]

('semanaaaa', 1)

In [0]:
stoi = {}

i = 0
for word_simbol, count in vocab_keys:
    stoi[word_simbol] = i
    i+=1
    
stoi['_end_'  ] = i
#stoi['_blank_'] = i + 1

In [14]:
stoi['que'],stoi['ja'],stoi['_end_']#, stoi['.'], stoi['_blank_']

(0, 1779, 19779)

In [15]:
print("Max key-value \t= ", len(stoi.keys())-1)

VOCAB_SIZE = len(stoi.keys())
print("Vocab Size \t= ", VOCAB_SIZE)

Max key-value 	=  19779
Vocab Size 	=  19780


In [0]:
itos = {}

for s, i in stoi.items():
    itos[i] = s

In [0]:
assert ('que'     == itos[stoi['que'    ]])
assert ('ja'      == itos[stoi['ja'     ]])
#assert ('.'       == itos[stoi['.'      ]])
assert ('_end_'   == itos[stoi['_end_'  ]])
#assert ('_blank_' == itos[stoi['_blank_']])

In [0]:
x_input = []
ls      = []

for sentence in sentences:
    i_sentence = []
    for word_simbol in sentence:
        i_sentence.append(stoi[word_simbol])
    i_sentence.append(stoi['_end_'])
    x_input.append(i_sentence)
    ls.append(len(i_sentence))

In [19]:
MAXLEN = max(ls)
print('MAXLEN :',MAXLEN)

MAXLEN : 50


In [0]:
#blank_value = stoi['_blank_']
#print('Blank Value = ', blank_value)

In [0]:
#from keras.preprocessing.sequence import pad_sequences

#x_train = pad_sequences(x_input, maxlen = MAXLEN, value=blank_value)

#print('Shape of data train tensor:', x_train.shape)

In [0]:
#x_input[0]

In [23]:
stoi['_end_']

19779

In [0]:
x_train_seq  = []
y_train_next = []

SEQ_LEN      = 3

for x_input_obs in x_input:
    MAXLEN = len(x_input_obs)
    for i in range(MAXLEN - SEQ_LEN):
        x_train_seq.append(x_input_obs[i:i+SEQ_LEN])
        y_train_next.append(x_input_obs[i+SEQ_LEN])

In [0]:
#for i in range(len(x_train_seq)):
#  print(x_train_seq[i], '->', y_train_next[i])

In [0]:
!pip install -q keras

In [0]:
#!pip3 install keras

In [28]:
#import tensorflow as tf
import keras 

#from keras import backend as K

#print(K.tensorflow_backend._get_available_gpus())


#config = tf.ConfigProto(allow_soft_placement = True, 
#                        device_count = {'CPU' : 1, 'GPU' : 0})

#sess = tf.Session(config=config)

#K.set_session(sess)

Using TensorFlow backend.


In [0]:
#from keras.utils import to_categorical
from keras.layers import LSTM, Input, Bidirectional, Dropout, Dense, Activation
from keras.models import Model

In [30]:
seq_input = Input(shape=(SEQ_LEN, VOCAB_SIZE))

rnn       = Bidirectional(LSTM(128, activation="relu"))(seq_input)
#rnn       = LSTM(128, activation="relu")(seq_input)
rnn       = Dropout(0.5)(rnn)
rnn       = Dense(VOCAB_SIZE)(rnn)
output    = Activation('softmax')(rnn)

model     = Model(inputs=[seq_input], outputs=[output])

model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['categorical_accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3, 19780)          0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256)               20386816  
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 19780)             5083460   
_________________________________________________________________
activation_1 (Activation)    (None, 19780)             0         
Total params: 25,470,276
Trainable params: 25,470,276
Non-trainable params: 0
_________________________________________________________________


In [0]:
import numpy as np

def X_to_OneHot(batch_sentences, num_classes):
  output = np.zeros((len(batch_sentences), SEQ_LEN, num_classes), dtype=np.bool)
  
  for i,sentence in enumerate(batch_sentences):
    for k,value in enumerate(sentence):
      output[i, k, value] = 1
  
  return output

def Y_to_OneHot(batch_label, num_classes):
  output = np.zeros((len(batch_label), num_classes), dtype=np.bool)
  
  for i, label in enumerate(batch_label):
    output[i, label] = 1
  
  return output  

In [0]:
#xp = X_to_OneHot(x_train_seq[:10], VOCAB_SIZE)
#yp = Y_to_OneHot(y_train_next[:10], VOCAB_SIZE)

In [0]:
#assert (np.argmax(xp[0,1,:]) == x_train_seq[0][1])
#assert (np.argmax(yp[0,:]) == y_train_next[0])

In [34]:
len(x_train_seq)

200482

In [0]:
model.load_weights(DRIVE_PATH+'model/textG_best_weights.02-7.3152.hdf5')

In [0]:
from keras.callbacks import ModelCheckpoint

BATCH_SIZE = 10000

filepath   = DRIVE_PATH + "model/textG_best_weights.{epoch:02d}-{val_loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=0, mode='auto', period=2)

n_epoch = 10

for ie in range(n_epoch):
  print("\nEPOCH : ", ie,end=' ===========================\n\n')
  for ibatch in range(0, len(x_train_seq), BATCH_SIZE):
      i_begin = ibatch
      i_end   = min(ibatch + BATCH_SIZE, len(x_train_seq))

      #print('BATCH', i_begin, '-',i_end)
      x_train_one_hot = X_to_OneHot( x_train_seq[i_begin:i_end], num_classes = VOCAB_SIZE)
      y_train_one_hot = Y_to_OneHot(y_train_next[i_begin:i_end], num_classes = VOCAB_SIZE)  
      
      model.fit(x_train_one_hot, y_train_one_hot, 
                batch_size=256, 
                shuffle=True, 
                epochs=2,
                callbacks = [checkpoint],
                validation_split=0.1)    


EPOCH :  0 ===========================

Train on 9000 samples, validate on 1000 samples
Epoch 1/2
9000/9000 [==============================] - 10s 1ms/step - loss: 3.7899 - categorical_accuracy: 0.2918 - val_loss: 6.4853 - val_categorical_accuracy: 0.1290
Epoch 2/2
9000/9000 [==============================] - 7s 744us/step - loss: 3.5189 - categorical_accuracy: 0.3181 - val_loss: 6.5338 - val_categorical_accuracy: 0.1390
Train on 9000 samples, validate on 1000 samples
Epoch 1/2
9000/9000 [==============================] - 7s 741us/step - loss: 3.8054 - categorical_accuracy: 0.2886 - val_loss: 6.3058 - val_categorical_accuracy: 0.1500
Epoch 2/2
9000/9000 [==============================] - 7s 763us/step - loss: 3.5345 - categorical_accuracy: 0.3218 - val_loss: 6.3574 - val_categorical_accuracy: 0.1490
Train on 9000 samples, validate on 1000 samples
Epoch 1/2
9000/9000 [==============================] - 7s 742us/step - loss: 3.8244 - categorical_accuracy: 0.2933 - val_loss: 6.8071 - val_

In [0]:
text = 'creo que ya'
seq  = [[stoi[word] for word in text.split()[-3:]]]

print(text, end=' ')
for i in range(10):
  val  = np.argmax(model.predict(X_to_OneHot(seq, VOCAB_SIZE)))
  if itos[val] == '_end_':
    break
  print(itos[val], end=' ')
  
  M = seq[0][1:]
  M.append(val)
  seq = [M]
  #print(seq)

creo que ya me voy a dormir 